In [19]:
import enum
from functools import total_ordering
import random

In [5]:
class State(enum.Enum):
    waiting = enum.auto() # called when the class is compiled
    started  = enum.auto()
    finished = enum.auto()

for member in State:
    print(member.name, member.value)

waiting 1
started 2
finished 3


In [7]:
class State(enum.Enum):
    waiting = 100 
    started  = enum.auto()
    finished = enum.auto()

for member in State:
    print(member.name, member.value)

waiting 100
started 101
finished 102


In [8]:
class State(enum.Enum):
    waiting = enum.auto() 
    started  = 1
    finished = enum.auto()
for member in State:
    print(member.name, member.value)

waiting 1
finished 2


started became alias to waiting

In [9]:
State.__members__

mappingproxy({'waiting': <State.waiting: 1>,
              'started': <State.waiting: 1>,
              'finished': <State.finished: 2>})

In [10]:
@enum.unique
class State(enum.Enum):
    waiting = enum.auto() 
    started  = 1
    finished = enum.auto()
for member in State:
    print(member.name, member.value)

ValueError: duplicate values found in <enum 'State'>: started -> waiting

how auto() works

In [11]:
hasattr(State, "_generate_next_value_")

True

In [16]:
class State(enum.Enum):
    def _generate_next_value_(name, start, count, last_values):
        print(name, start, count, last_values)
        return 100
    
    waiting = enum.auto() 
    started  = enum.auto()
    finished = enum.auto()
print()
print(State.__members__)
print()
print(list(State))

waiting 1 0 []
started 1 1 [100]
finished 1 2 [100, 100]

OrderedDict([('waiting', <State.waiting: 100>), ('started', <State.waiting: 100>), ('finished', <State.waiting: 100>)])

[<State.waiting: 100>]


In [17]:
class State(enum.Enum):
    def _generate_next_value_(name, start, count, last_values):
        print(name, start, count, last_values)
        return 100
    
    waiting = enum.auto() 
    started  = enum.auto()
    с = 200
    finished = enum.auto()
print()
print(State.__members__)
print()
print(list(State))

waiting 1 0 []
started 1 1 [100]
finished 1 3 [100, 100, 200]

OrderedDict([('waiting', <State.waiting: 100>), ('started', <State.waiting: 100>), ('с', <State.с: 200>), ('finished', <State.waiting: 100>)])

[<State.waiting: 100>, <State.с: 200>]


In [20]:
random.seed(0)

In [22]:
class State(enum.Enum):
    def _generate_next_value_(name, start, count, last_values):
        while True:
            new_value = random.randint(1, 100)
            if new_value not in last_values:
                return new_value
        
    
    a = enum.auto() 
    b = enum.auto()
    с = enum.auto()
    d = enum.auto()
    
for member in State:
    print(member.name, member.value)

a 50
b 98
с 54
d 6


In [23]:
class State(enum.Enum):
    def _generate_next_value_(name, start, count, last_values):
        return name.title()
    ready = enum.auto()
    waiting = enum.auto()
    finished = enum.auto()
for member in State:
    print(member.name, member.value)

ready Ready
waiting Waiting
finished Finished


inheritance

In [24]:
class NameAsString(enum.Enum):
    def _generate_next_value_(name, start, count, last_values):
        return name.lower()
class Enum1(NameAsString):
    A = enum.auto()
    B = enum.auto()
list(Enum1)

[<Enum1.A: 'a'>, <Enum1.B: 'b'>]

In [25]:
class Enum2(NameAsString):
    ready = enum.auto()
    waiting = enum.auto()
    finished = enum.auto()
list(Enum2)

[<Enum2.ready: 'ready'>,
 <Enum2.waiting: 'waiting'>,
 <Enum2.finished: 'finished'>]

#### how to not use values (if they are subjects to change), but only names

In [26]:
class State(enum.Enum):
    ready = object()
    waiting = object()
    finished = object()

In [27]:
State.ready, State.waiting

(<State.ready: <object object at 0x00000240ACBA75A0>>,
 <State.waiting: <object object at 0x00000240ACBA7530>>)

In [28]:
class Aliased(enum.Enum):
    def _generate_next_value_(name, start, count, last_values):
        print(f"count={count}") # count include the aliases
        if count % 2 == 1:
            #make this member an alias of the previous one
            return last_values[-1]
        else:
            return last_values[-1] + 1
        
    green = 1
    green_alias = 1
    red = 10
    crimson = enum.auto()
    blue = enum.auto()
    aqua = enum.auto()

count=3
count=4
count=5


In [30]:
Aliased.__members__

mappingproxy({'green': <Aliased.green: 1>,
              'green_alias': <Aliased.green: 1>,
              'red': <Aliased.red: 10>,
              'crimson': <Aliased.red: 10>,
              'blue': <Aliased.blue: 11>,
              'aqua': <Aliased.blue: 11>})

In [31]:
list(Aliased)

[<Aliased.green: 1>, <Aliased.red: 10>, <Aliased.blue: 11>]

In [32]:
class Aliased(enum.Enum): # always create aliases
    def _generate_next_value_(name, start, count, last_values):
        return last_values[-1]
       
   

In [33]:
class Color(Aliased):
    red = object()
    crimson = enum.auto()
    carmine = enum.auto()
    
    blue = object()
    aquamarine = enum.auto()
    azure = enum.auto()

In [34]:
Color.__members__

mappingproxy({'red': <Color.red: <object object at 0x00000240ACBA7640>>,
              'crimson': <Color.red: <object object at 0x00000240ACBA7640>>,
              'carmine': <Color.red: <object object at 0x00000240ACBA7640>>,
              'blue': <Color.blue: <object object at 0x00000240ACBA79C0>>,
              'aquamarine': <Color.blue: <object object at 0x00000240ACBA79C0>>,
              'azure': <Color.blue: <object object at 0x00000240ACBA79C0>>})

In [35]:
list(Color)

[<Color.red: <object object at 0x00000240ACBA7640>>,
 <Color.blue: <object object at 0x00000240ACBA79C0>>]

In [36]:
Color.azure

<Color.blue: <object object at 0x00000240ACBA79C0>>

In [37]:
Color.azure is Color.blue

True